# Первичная обработка данных

Поскольку данные из-за многочисленных ограничений VK API собирались в несколько этапов, после сбора необходимо было провести некоторую предобработку: объединить посты и комментарии за разные месяцы, удалить дубликаты, создать единый массив данных, где по строчкам будет комментарий - пост, под которым комментарий был написан.

In [ ]:
import polars as pl
import pandas as pd
import orjsonl
import datetime
import matplotlib.pyplot as plt

# Собираем посты вместе

In [ ]:
with open('posts_pol_10_11.json', 'r', encoding='utf-8') as f:
    posts1 = pd.read_json(f, lines=True)
    posts1 = posts1.drop_duplicates(subset=['date', 'from_id', 'id', 'owner_id', 'text'], keep='first')

with open('posts_pol_12_1.json', 'r', encoding='utf-8') as f:
    posts2 = pd.read_json(f, lines=True)
    posts2 = posts2.drop_duplicates(subset=['date', 'from_id', 'id', 'owner_id', 'text'], keep='first')

with open('posts_pol_2_3_filtered.json', 'r', encoding='utf-8') as f:
    posts3 = pd.read_json(f, lines=True)
    posts3 = posts3.drop_duplicates(subset=['date', 'from_id', 'id', 'owner_id', 'text'], keep='first')

posts = pd.concat([posts1, posts2, posts3])
posts = posts.drop_duplicates(subset=['date', 'from_id', 'id', 'owner_id', 'text'], keep='first')

In [ ]:
# Для ускорения работы и меньшего объема будет использовать библиотеку polars

posts_pl = pl.DataFrame(posts)
posts_pl.write_parquet("posts_pol_10_3.parquet")

# Файл в формате parquet весит в 10 раз меньше!

# Собираем комментарии вместе

In [ ]:
with open('comm_pol_10_11.json', 'r', encoding='utf-8') as f:
    comm1 = pd.read_json(f, lines=True)
    comm1 = comm1.drop_duplicates(subset=['date', 'from_id', 'id', 'owner_id', 'text'], keep='first')

with open('comm_pol_12_1.json', 'r', encoding='utf-8') as f:
    comm2 = pd.read_json(f, lines=True)
    comm2 = comm2.drop_duplicates(subset=['date', 'from_id', 'id', 'owner_id', 'text'], keep='first')

with open('comm_pol_2_3.json', 'r', encoding='utf-8') as f:
    comm3 = pd.read_json(f, lines=True)
    comm3 = comm3.drop_duplicates(subset=['date', 'from_id', 'id', 'owner_id', 'text'], keep='first')


comm = pd.concat([comm1, comm2, comm3])
comm = comm.drop_duplicates(subset=['date', 'from_id', 'id', 'owner_id', 'text'], keep='first')

In [ ]:
comm_pl = pl.DataFrame(c)
comm_pl.write_parquet("comm_pol_10_3.parquet")

# Объединяем посты и комментарии в один массив данных

In [ ]:
comm = pl.read_parquet("comm_pol_10_3.parquet")
posts = pl.read_parquet("posts_pol_10_3.parquet")

In [ ]:
comm_with_posts = comm_pl.join(posts_pl, how='inner', left_on=['post_id', 'owner_id'], right_on=['id', 'owner_id'])
comm_with_posts = comm_with_posts.rename({'text_right': 'post_text'})
comm_with_posts = comm_with_posts.drop('date_right')

comm_with_posts.write_parquet("comm_with_posts_final_10_3.parquet")